**MASK WEAR DETECTION WITH LOGISTIC REGRESSION :**

In [3]:
from torch.utils.data import DataLoader
import os
import cv2
import random

class DataSet :

    def __init__(self,root,labels):
        """Init function should not do any heavy lifting, but
            must initialize how many items are available in this data set.
        """

        self.ROOT = root
        self.images = read_images(root)
        self.labels = labels

    def __len__(self):
        """return number of elements in our dataset"""
        
        return len(self.images)

    def __getitem__(self, idx):
        """ Here we have to return the item requested by `idx`
            The PyTorch DataLoader class will use this method to make an iterable for
            our training or validation loop.
        """

        img = self.images[idx]
        label = self.labels

        return img, label
    
    def __create_training_and_test_dataset__(self,dataset) :
        """THAT S A BAD IDEA
        """
        ds_uwm = DataSet(self.ROOT,self.labels)
        ds_wm = DataSet(dataset.ROOT,dataset.labels)
        ds_train = []
        ds_test = []
        i,j = 0,0
        for img1,lbl1 in ds_uwm :
            for img2,lbl2 in ds_wm :
                while i<len(ds_uwm) and j<len(ds_wm) :
                    random1 = random.randint(0,1)
                    random2 = random.randint(0,1)
                    if random1 == 0 :
                        # We add uwm in train
                        ds_train.append((img1,lbl1))
                    elif random1 == 1 :
                        # We add uwm in test
                        ds_train.append((img1,lbl1))
                    elif random2 == 0 :
                        # We add wm in train
                        ds_train.append((img2,lbl2))
                    else :
                        # We add wm in test
                        ds_train.append((img2,lbl2))
                

                    
            
            
        
        

In [5]:
import torch
import numpy as np

def read_images(root) :
        '''load all the images from a folder'''
        images = []
        for filename in os.listdir(root) :
            '''
            Python method listdir() returns a list containing the names of the 
            entries in the directory given by path. The list is in arbitrary order. 
            It does not include the special entries '.' and '..' even if they are 
            present in the directory.'''
            img = cv2.imread(os.path.join(root,filename))
            if img is not None:
                img = img[:,:,::-1]/255 # To shuffle the color channels form BGR to RGB and /255 to normalize
                # A REGARDER COMMENT CA MARCHE PLUS EN DETAIL 
                images.append(img)
        return torch.from_numpy(np.array(images))

ds_uwm = DataSet('U',0)
ds_wm = DataSet("W",1)

In [7]:
img, label = ds_uwm[0]
print(img,label)
print(type(img))
print(img.shape)
print(len(ds_uwm),len(ds_wm))

tensor([[[0.7608, 0.0000, 0.0000],
         [0.7647, 0.0000, 0.0000],
         [0.7686, 0.0000, 0.0000],
         ...,
         [0.2824, 0.2902, 0.1137],
         [0.3216, 0.3216, 0.1176],
         [0.3647, 0.3608, 0.1294]],

        [[0.7608, 0.0000, 0.0039],
         [0.7686, 0.0000, 0.0039],
         [0.7765, 0.0000, 0.0000],
         ...,
         [0.3176, 0.3216, 0.1451],
         [0.3529, 0.3529, 0.1490],
         [0.3922, 0.3843, 0.1529]],

        [[0.7686, 0.0000, 0.0039],
         [0.7765, 0.0000, 0.0039],
         [0.7804, 0.0000, 0.0000],
         ...,
         [0.3137, 0.3373, 0.1569],
         [0.3647, 0.3804, 0.1725],
         [0.4157, 0.4235, 0.1882]],

        ...,

        [[0.7176, 0.0510, 0.6078],
         [0.7137, 0.0392, 0.5961],
         [0.7216, 0.0549, 0.6039],
         ...,
         [0.9961, 0.6627, 0.5176],
         [1.0000, 0.6510, 0.4980],
         [0.9961, 0.6549, 0.5059]],

        [[0.7255, 0.0431, 0.6039],
         [0.7137, 0.0275, 0.5882],
         [0.

# Training and Validation Datasets

While building real world machine learning models, it is quite common to split the dataset into 3 parts:

1. **Training set** - used to train the model i.e. compute the loss and adjust the weights of the model using gradient descent.
2. **Validation set** - used to evaluate the model while training, adjust hyperparameters (learning rate etc.) and pick the best version of the model.
3. **Test set** - used to compare different models, or different types of modeling approaches, and report the final accuracy of the model.

In [5]:
import torch.nn as nn

input_size = 1024*1024
num_classes = 2

# Logistic regression model
model = nn.Linear(input_size, num_classes)

In [6]:
print(model.weight.shape)
model.weight

torch.Size([2, 1048576])


Parameter containing:
tensor([[-7.4185e-04, -6.2460e-04,  4.1101e-04,  ..., -5.1341e-04,
          1.1804e-04, -5.4645e-06],
        [-1.1196e-04,  1.9617e-04, -5.4346e-06,  ..., -4.4239e-04,
          6.9568e-05,  1.3496e-04]], requires_grad=True)

In [7]:
print(model.bias.shape)
model.bias

torch.Size([2])


Parameter containing:
tensor([ 0.0009, -0.0001], requires_grad=True)

In [8]:
for images, labels in ds:
    print(labels)
    print(images.shape)
    outputs = model(images)
    break

0
torch.Size([1024, 1024, 3])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1048576x3 and 1048576x2)

In [9]:
class MWDModel(nn.Module): # MWD for Mask Wear detection
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, xb):
        xb = xb.reshape(-1, 1048576)
        out = self.linear(xb)
        return out
    
model = MWDModel()

In [10]:
print(model.linear.weight.shape, model.linear.bias.shape)
list(model.parameters())

torch.Size([2, 1048576]) torch.Size([2])


[Parameter containing:
 tensor([[-0.0001,  0.0004,  0.0010,  ..., -0.0007, -0.0006,  0.0009],
         [-0.0003,  0.0005,  0.0008,  ...,  0.0002,  0.0006,  0.0009]],
        requires_grad=True),
 Parameter containing:
 tensor([ 6.1020e-05, -4.8714e-04], requires_grad=True)]

In [13]:
for images, labels in ds:
    outputs = model(images.float())
    break

print('outputs.shape : ', outputs.shape)
print('Sample outputs :\n', outputs[:2].data)

'''
When the error is RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1', you 
would need to use the .float() function since it says Expected object of scalar type Float.

Therefore, the solution is changing y_pred = model(X_trainTensor) to y_pred = model(X_trainTensor.float()).

Likewise, when you get another error for loss = loss_fn(y_pred, y_trainTensor), you need y_trainTensor.long() since the 
error message says Expected object of scalar type Long.

You could also do model.double()'''

outputs.shape :  torch.Size([3, 2])
Sample outputs :
 tensor([[ 0.3172, -0.0013],
        [ 0.0325, -0.4738]])


In [14]:
import torch.nn.functional as F

In [15]:
# Apply softmax for each output row
probs = F.softmax(outputs, dim=1)

# Look at sample probabilities
print("Sample probabilities:\n", probs[:2].data)

# Add up the probabilities of an output row
print("Sum: ", torch.sum(probs[0]).item())

Sample probabilities:
 tensor([[0.5789, 0.4211],
        [0.6239, 0.3761]])
Sum:  1.0


In [16]:
max_probs, preds = torch.max(probs, dim=1)
print(preds)
print(max_probs)

tensor([0, 0, 0])
tensor([0.5789, 0.6239, 0.6068], grad_fn=<MaxBackward0>)


In [17]:
labels

0

In [19]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [20]:
accuracy(outputs, labels)

tensor(1.)

In [21]:
loss_fn = F.cross_entropy

In [22]:
# Loss for current batch of data
loss = loss_fn(outputs, labels)
print(loss)

AttributeError: 'int' object has no attribute 'size'